In [1]:
from IPython.display import display
from os.path import isfile, join, exists
from os import listdir, makedirs, walk, remove, getlogin
from nilearn import image, plotting
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import fileinput

/jukebox/pkgs/ANACONDAPY/ni/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#---- Set subject specific variables ----#

# Set subject ID
subid = 'impr_g1_s3_020518'

# manually set scan num, type, and conds
epis = [9, 10, 11, 12]
anat = [8]
fieldmaps = [13, 14]
epi_cond_order = ['neg1', 'pos1', 'mixed1']

scanNum = epis + anat + fieldmaps 
scanType = ['epi', 'epi', 'epi', 'epi', 'anat', 'fieldmap', 'fieldmap']
scanCond = epi_cond_order + ['pieman', 'anat', 'se_fieldmap_ap', 'se_fieldmap_pa' ]

ref_scan = 'pieman'
conds_order_std = ['pos1', 'neg1', 'mixed1']

In [3]:
#---- Set experiment-specific parameters ----#

# Set data dir
projdir = '/mnt/bucket/labs/hasson/mai/projects/impressions/data'

# Set code dir
codedir = '/mnt/bucket/labs/hasson/mai/projects/impressions/code/preproc/bash'

# Set template dir
templatedir = '/mnt/bucket/labs/hasson/mai/projects/impressions/code/preproc/template'

In [4]:
#---- Setup subject directories -----#

subdir = join(projdir, subid)
if not exists(subdir):
    print(subdir + ' does not exist, creating')
    makedirs(subdir)
    
subdir_data = join(subdir, 'raw')
if not exists(subdir_data):
    print(subdir_data + ' does not exist, creating')
    makedirs(subdir_data)

subdir_log = join(subdir, 'logs')
if not exists(subdir_log):
    print(subdir_log + ' does not exist, creating')
    makedirs(subdir_log)
    
preproc_script_dir = join(subdir, 'preproc', 'scripts')
if not exists(preproc_script_dir):
    print(preproc_script_dir + ' does not exist, creating')
    makedirs(preproc_script_dir)


In [ ]:
#---- Copy and modify preproc scripts for subject ----#

# Setup script1: 01_convert_dcm2nii.py
template_script = join(templatedir, '01_convert_dcm2nii.py')
subj_script = join(preproc_script_dir, '01_convert_dcm2nii.py')
!cp {template_script} {subj_script}

for line in fileinput.input(subj_script, inplace=1):
    line = re.sub('###SUB_ID###', "'" + subid + "'", line.rstrip())
    line = re.sub('###EPI_NS###', str(epis), line.rstrip())
    line = re.sub('###ANAT_N###', str(anat), line.rstrip())
    line = re.sub('###FIELDMAP_NS###', str(fieldmaps), line.rstrip())
    line = re.sub('###EPI_CONDS###', str(epi_cond_order), line.rstrip())
    line = re.sub('###SCAN_TYPE###', str(scanType), line.rstrip())
    line = re.sub('###SCAN_COND###', str(scanCond), line.rstrip())
    print(line)
fileinput.close()

# Setup script: 02_make_preprocFiles.py
template_script = join(templatedir, '02_make_preprocFiles.py')
subj_script = join(preproc_script_dir, '02_make_preprocFiles.py')
!cp {template_script} {subj_script}

for line in fileinput.input(subj_script, inplace=1):
    line = re.sub('###SUB_ID###', "'" + subid + "'", line.rstrip())
    line = re.sub('###REF_SCAN###', "'" + ref_scan + "'", line.rstrip())
    print(line)
fileinput.close()

# Setup script: 03_run_preproc.py
template_script = join(templatedir, '03_run_preproc.py')
subj_script = join(preproc_script_dir, '03_run_preproc.py')
!cp {template_script} {subj_script}

for line in fileinput.input(subj_script, inplace=1):
    line = re.sub('###SUB_ID###', "'" + subid + "'", line.rstrip())
    line = re.sub('###CONDS###', str(conds_order_std), line.rstrip())
    print(line)
fileinput.close()


In [7]:
fileinput.close()